# GPT Model w/o Descriptions

In [2]:
import pandas as pd
from datasets import Dataset
import ast
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
import os
import csv

/sfs/gpfs/tardis/home/nuf8ms/Documents/MSDS/LLM/DS6051-Project/test/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.chdir('..')
os.getcwd()

'/sfs/gpfs/tardis/home/nuf8ms/Documents/MSDS/LLM'

In [3]:
df2 = pd.read_csv('DS6051-Project/Playlist_data_with_lyrics.csv')
df3 = pd.read_csv('DS6051-Project/playlist_data.csv')
df = pd.read_csv('Additional Data/Combined_Songs_Artists.csv')

In [4]:
import pandas as pd
from datasets import Dataset
import ast

# Load and parse the dataset

# Convert stringified lists to actual lists
# for col in ['Playlist_Songs', 'Playlist_Artists']:
#     df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

df["Playlist_Songs"] = df["Playlist_Songs"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

## Training Targets 1st Iteration, 3 epoch Loss ~ 3, 5 epoch Loss ~ 2.5
## playlist_data.csv
# Create prompt + output text for GPT-2 training
# def format_example(row):
#     song_lines = [f"{s} - {a}" for s, a in zip(row['Playlist_Songs'], row['Playlist_Artists'])]
#     return f"### Prompt: {row['Playlist_Name']}\n### Playlist:\n" + "\n".join(song_lines)

## Training Targets 2nd Iteration, 5 epoch Loss ~ 1.3
## playlist_data.csv
# def format_example(row):
#     lines = [f"[SONG] {s} [ARTIST] {a}" for s, a in zip(row["Playlist_Songs"], row["Playlist_Artists"])]
#     return f"### Prompt: {row['Playlist_Name']}\n### Playlist:\n" + "\n".join(lines)

## Training targets 3rd Iteration, 5 epoch Loss ~ 1.35
## playlist_data.csv
# def format_example(row):
#     lines = [f"[SONG] {song} [ARTIST] {artist}" for song, artist in zip(row["Playlist_Songs"], row["Playlist_Artists"])]
#     playlist_body = "\n".join(lines)
    
#     # Include Playlist_Description in the prompt for training only
#     return (
#         f"### Prompt: {row['Playlist_Name']}\n"
#         f"### Description: {row['Playlist_Description']}\n"
#         f"### Playlist:\n{playlist_body}"
#     )

## Training targets 4th Iteration, 
## Combined_Songs_Artists.csv
def format_example(row):
    # Use the song-artist strings as-is
    playlist_body = "\n".join(row["Playlist_Songs"])
    return (
        f"### Prompt: {row['Playlist_Name']}\n"
        f"### Description: {row['Playlist_Description']}\n"
        f"### Playlist:\n{playlist_body}"
    )

df['text'] = df.apply(format_example, axis=1)

# Convert to Hugging Face Dataset
dataset = Dataset.from_dict({"text": df['text'].tolist()})

In [5]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 doesn't have a pad token

model = GPT2LMHeadModel.from_pretrained("gpt2-medium")
model.resize_token_embeddings(len(tokenizer))  # In case we add special tokens

# Tokenize
def tokenize(batch):
    encodings = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)
    encodings["labels"] = encodings["input_ids"].copy()  # 🔥 Add this line
    return encodings
    
tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map: 100%|██████████| 2289/2289 [00:11<00:00, 203.23 examples/s]


In [6]:
from transformers import Trainer, TrainingArguments
import torch

training_args = TrainingArguments(
    output_dir="./DS6051-Project/models/gpt2_Combined_Song_Artists_Checkpoints",
    overwrite_output_dir=True,
    num_train_epochs=30,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=50,
    fp16=True if torch.cuda.is_available() else False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

trainer.train()

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,3.375100
100,2.921100
150,2.866300
200,2.784800
250,2.696400
300,2.689000
350,2.692500
400,2.557700
450,2.561700
500,2.577100


TrainOutput(global_step=1145, training_loss=2.6366436954148473, metrics={'train_runtime': 100.2579, 'train_samples_per_second': 22.831, 'train_steps_per_second': 11.421, 'total_flos': 2125795889774592.0, 'train_loss': 2.6366436954148473, 'epoch': 1.0})

In [7]:
model.save_pretrained("./DS6051-Project/models/gpt2_Combined_Song_Artists")

In [9]:
tokenizer.save_pretrained("./DS6051-Project/models/gpt2_Combined_Song_Artists")

('./DS6051-Project/models/gpt2_Combined_Song_Artists/tokenizer_config.json',
 './DS6051-Project/models/gpt2_Combined_Song_Artists/special_tokens_map.json',
 './DS6051-Project/models/gpt2_Combined_Song_Artists/vocab.json',
 './DS6051-Project/models/gpt2_Combined_Song_Artists/merges.txt',
 './DS6051-Project/models/gpt2_Combined_Song_Artists/added_tokens.json')

In [8]:
print(type(model))

<class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'>


In [19]:
def generate_playlist(prompt, max_length=200):
    input_text = f"### Prompt: {prompt}\n### Playlist:\n"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    output = model.generate(
        input_ids = input_ids.to('cuda'),
        max_length=max_length,
        temperature=0.9,
        top_p=0.95,
        do_sample=True,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    )

    result = tokenizer.decode(output[0], skip_special_tokens=True)
    return result.split("### Playlist:\n")[1].strip()

# Try it out!
prompt = "Road Trip with Classic Rock"
print(generate_playlist(prompt))

/sfs/gpfs/tardis/home/nuf8ms/Documents/MSDS/LLM/DS6051-Project/test/lib/python3.11/site-packages/transformers/generation/utils.py:2345: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cuda, whereas the model is on cpu. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cpu') before running `.generate()`.
  warnings.warn(


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [18]:
## Import Saved Model and use with generate_playlist function
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained("./DS6051-Project/models/gpt2_Combined_Song_Artists", local_files_only = True)
tokenizer = GPT2Tokenizer.from_pretrained("./DS6051-Project/models/gpt2_Combined_Song_Artists", local_files_only = True)

tokenizer.pad_token = tokenizer.eos_token

In [4]:
os.getcwd()

'/sfs/gpfs/tardis/home/nuf8ms/Documents/MSDS/LLM/DS6051-Project'

In [6]:
os.path.join()

['checkpoint-5725', 'checkpoint-5500']

# GPT Model w/ Descriptions

In [6]:
print(os.path.exists("./models/gpt2_Combined_Song_Artists"))
print(os.listdir("./models/gpt2_Combined_Song_Artists"))

True
['model.safetensors', 'config.json', 'vocab.json', 'tokenizer_config.json', 'special_tokens_map.json', 'generation_config.json', 'merges.txt']


In [21]:
model.to('cuda')
def generate_playlist(prompt, max_length=200):
    input_text = f"### Prompt: {prompt}\n### Playlist:\n"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    output = model.generate(
        input_ids = input_ids.to('cuda'),
        max_length=max_length,
        temperature=0.9,
        top_p=0.95,
        do_sample=True,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    )

    result = tokenizer.decode(output[0], skip_special_tokens=True)
    return result.split("### Playlist:\n")[1].strip()

# Try it out!
prompt = "Road Trip with Classic Rock"
print(generate_playlist(prompt))

Livin' On A Prayer : Jimi Hendrix
American Girl : Billy Joel
When I Come Around : The All-American Rejects
I Can't Help Falling in Love with You : Tears For Fears
You Make My Dreams (Come True) : Lynyrd Skynyrd
Hey Jude : TOTO
Can't Help Falling In Love with You : Tears For Fears
I Ain't Takin' It So Easy : The All-American Rejects
The Sweetest Sound : Aerosmith
Black Dog : Dire Straits
I'm Still Standing : Guns N' Roses
Heart-Shaped Box : Eagles
Thunderstruck : Lynyrd Skynyrd
Rock Your Body : The All-American Rejects
Shake It for Me : Dire Straits
Don't You Worry Child : Starship
You Make My Dreams (Come True
